In this project, you will create a deep learning model that detects the TNt tubes formed in between cells. Below is a brief description of what TNTs are:

"Tunneling nanotubes (TNTs) are elongated structures extending from and connecting cancer cell membranes. They permit the exchange of molecules, vesicles, and mitochondria, as well as genetic and metabolic signals that promote carcinogenesis. Given that they permit intercellular trafficking and communication, TNTs may serve as an important imaging biomarker of cancer cell response vs. resistance to therapy. On fluorescence imaging of cancer cell cultures, TNTs appear to be no thicker than 1 µm and vary in length from 10 to 100+ µm. TNTs can be spotted by a trained eye, but using human experts to obtain an accurate count and location of TNTs is a time-intensive process. A precise quantitative analysis of TNTs could aid in the objective assessment of cancer response to various therapeutic interventions."

In this project, the original images were created by taking a grid of 5 × 5 tiled images, each measuring 1388 × 1040 pixels, and then stitching them together. This process resulted in shadows along the stitched edges, which significantly degraded the model performance at later stages. You may start from removing these shadows. To remove those shadows, you may use BaSiC, an image correction method for background and shading correction for image sequences, available as a Fiji/ImageJ. You may consider other packages/lobraries for this purpose. You may also consider using different filters. You may check the following link: https://www.youtube.com/watch?v=xCHbcVUCYBI. You may find very useful short videos in that channel for image processing.

After preprocessing, you may want to divide the original image into smaller pieces. The original image in the training dataset was stitched together resulting in an image - size of 6283 × 4687 pixels. You can scan the images  with a sliding window of 512 × 512 pixels with a stride of 10 pixels, extracting patches containing the TNT regions using a bounding box. You may write a function that takes the window size as input. You may later create smaller images from 512x512 images using the same function.

Once you form training and test datasets from smaller images, you may simply train a VGG model. You may try different models. you goal is to find the images that contain TNTs. You may create multiple models that are trained with images with different sizes.

# Image Preprocessing

In [37]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# Define consistent paths
import os
import shutil
from PIL import Image
import numpy as np
import cv2
from tqdm import tqdm

# Paths
base_drive_path = "/content/drive/MyDrive/Mentorship_DL_Module/Second Projects/TNT Project"
base_colab_path = "/content/TNT_Project"


This is the image with labels

![Screen Shot 2022-04-30 at 5.32.27 AM.png](attachment:27b423ed-9de7-4634-9288-52f227b53968.png)

This is the same image without labels.

![Screen Shot 2022-04-30 at 5.33.17 AM.png](attachment:a426a0b5-5596-4037-a92b-24753646b87a.png)

- Create folders to save the images that are created by splitting the main image into smaller pieces


In [39]:
# Create necessary directories
labeled_dir = os.path.join(base_colab_path, "labeled")
os.makedirs(os.path.join(labeled_dir, "1"), exist_ok=True)  # TNT patches
os.makedirs(os.path.join(labeled_dir, "0"), exist_ok=True)  # No-TNT patches

In [40]:
# Copy images from Drive
shutil.copy(os.path.join(base_drive_path, "m05.png"), base_colab_path)
shutil.copy(os.path.join(base_drive_path, "m05-label.png"), base_colab_path)


'/content/TNT_Project/m05-label.png'

In [41]:
# Load images
image_path = os.path.join(base_colab_path, "m05.png")  # Unlabeled image
label_image_path = os.path.join(base_colab_path, "m05-label.png")  # Labeled image

pil_image = Image.open(image_path).convert("L")
image = np.array(pil_image)
label_image = np.array(Image.open(label_image_path).convert("L"))

![Screen Shot 2022-04-30 at 5.39.51 AM.png](attachment:323c38bf-351c-4123-a434-622843c08e15.png)

- Create smaller pieces from the main image by moving a window along the image. Mark the pieces with TNTs as 1 and all others as 0. This will give you the labeled images for your classification model.

In [42]:
# Define patch size and stride
window_size, stride = 512, 512
height, width = image.shape
patch_count = 0

In [43]:
# Define patch size and stride
window_size, stride = 512, 512  # You might want to experiment with different values
height, width = image.shape
patch_count = 0

# Extract and label patches in a single loop
for y in tqdm(range(0, height - window_size + 1, stride)):
    for x in range(0, width - window_size + 1, stride):
        # Extract patch from main image
        patch = image[y:y+window_size, x:x+window_size]

        # Extract corresponding label patch
        label_patch = label_image[y:y+window_size, x:x+window_size]

        # Calculate the percentage of white pixels (TNTs are white in the label image)
        white_pixel_percentage = np.sum(label_patch == 255) / label_patch.size * 100

        # Adjust the threshold as needed - This value might need fine-tuning
        label_folder = "1" if white_pixel_percentage > 2 else "0"

        # Save patch directly to labeled folder
        patch_filename = f"patch_{patch_count:06d}.png"
        patch_path = os.path.join(labeled_dir, label_folder, patch_filename)
        cv2.imwrite(patch_path, patch)

        patch_count += 1

print(f"✅ Successfully extracted and labeled {patch_count} patches!")


100%|██████████| 9/9 [00:00<00:00, 14.39it/s]

✅ Successfully extracted and labeled 108 patches!


In [44]:
print(train_gen.class_indices)  # Should output: {'0': 0, '1': 1}
print(val_gen.class_indices)    # Should output: {'0': 0, '1': 1}


{'0': 0, '1': 1}
{'0': 0, '1': 1}


In [45]:
print("Training class distribution:", dict(zip(*np.unique(train_gen.classes, return_counts=True))))
print("Validation class distribution:", dict(zip(*np.unique(val_gen.classes, return_counts=True))))


Training class distribution: {1: 87}
Validation class distribution: {1: 21}


In [46]:
import random
import shutil
import os

# Define paths
tnt_dir = os.path.join(labeled_dir, "1")
no_tnt_dir = os.path.join(labeled_dir, "0")

# Get file lists
tnt_files = os.listdir(tnt_dir)
no_tnt_files = os.listdir(no_tnt_dir)

# Ensure we have both classes
if len(no_tnt_files) == 0:
    raise ValueError("No No-TNT patches found! Ensure extracted both TNT and No-TNT patches.")

# Balance the dataset before splitting
num_patches = min(len(tnt_files), len(no_tnt_files))  # Get min count to balance
tnt_files = random.sample(tnt_files, num_patches)
no_tnt_files = random.sample(no_tnt_files, num_patches)

print(f"✅ Balanced dataset: {num_patches} TNT patches, {num_patches} No-TNT patches")

# Move extra files to a backup folder instead of deleting them
backup_dir = os.path.join(base_colab_path, "backup")
os.makedirs(backup_dir, exist_ok=True)

for f in os.listdir(tnt_dir):
    if f not in tnt_files:
        shutil.move(os.path.join(tnt_dir, f), os.path.join(backup_dir, f))

for f in os.listdir(no_tnt_dir):
    if f not in no_tnt_files:
        shutil.move(os.path.join(no_tnt_dir, f), os.path.join(backup_dir, f))

print(f"✅ After balancing: {len(tnt_files)} TNT patches, {len(no_tnt_files)} No-TNT patches")


✅ Balanced dataset: 108 TNT patches, 108 No-TNT patches
✅ After balancing: 108 TNT patches, 108 No-TNT patches


In [47]:
# Train VGG16 Model
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC
from tensorflow.keras.metrics import Precision, Recall
batch_size, image_size = 8, (224, 224)



In [48]:
# Data generators
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_gen = datagen.flow_from_directory(
    labeled_dir,
    target_size=(224, 224),
    batch_size=8,
    class_mode='binary',
    subset='training'  # Training data
)
val_gen = datagen.flow_from_directory(
    labeled_dir,
    target_size=(224, 224),
    batch_size=8,
    class_mode='binary',
    subset='validation'  # Validation data
)

Found 174 images belonging to 2 classes.
Found 42 images belonging to 2 classes.


In [49]:
print("Training class distribution:", dict(zip(*np.unique(train_gen.classes, return_counts=True))))
print("Validation class distribution:", dict(zip(*np.unique(val_gen.classes, return_counts=True))))


Training class distribution: {0: 87, 1: 87}
Validation class distribution: {0: 21, 1: 21}


In [50]:
# Model Setup
vgg_model = VGG16(include_top=False, pooling='avg', weights='imagenet', input_shape=(224, 224, 3))
for layer in vgg_model.layers[:-2]: layer.trainable = False

x = Flatten()(vgg_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=vgg_model.input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])

In [51]:
# Train with early stopping
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint(os.path.join(base_colab_path, "best_tnt_model.keras"), save_best_only=True)
]
model.fit(train_gen, validation_data=val_gen, epochs=3, steps_per_epoch=250, validation_steps=50, callbacks=callbacks)

Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 22/250 ━━━━━━━━━━━━━━━━━━━━ 17:46 5s/step - accuracy: 0.4529 - loss: 0.7297 - precision: 0.5099 - recall: 0.5679

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


250/250 ━━━━━━━━━━━━━━━━━━━━ 132s 499ms/step - accuracy: 0.4854 - loss: 0.7159 - precision: 0.4900 - recall: 0.4798 - val_accuracy: 0.5000 - val_loss: 0.6974 - val_precision: 0.5000 - val_recall: 0.0476
Epoch 2/3
250/250 ━━━━━━━━━━━━━━━━━━━━ 130s 497ms/step - accuracy: 0.4599 - loss: 0.7537 - precision: 0.4508 - recall: 0.3833 - val_accuracy: 0.5000 - val_loss: 0.6954 - val_precision: 0.5000 - val_recall: 0.9524
Epoch 3/3
250/250 ━━━━━━━━━━━━━━━━━━━━ 138s 491ms/step - accuracy: 0.5052 - loss: 0.7272 - precision: 0.5074 - recall: 0.7038 - val_accuracy: 0.5000 - val_loss: 0.7222 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00


In [54]:
# Prediction on new images
model = tf.keras.models.load_model(os.path.join(base_colab_path, "best_tnt_model.keras"))

In [55]:
# Predict example image
import random
test_image_path = os.path.join(base_colab_path, "labeled", "1", random.choice(os.listdir(os.path.join(labeled_dir, "1"))))
test_img = cv2.imread(test_image_path)
test_img = cv2.resize(test_img, (224, 224)).astype('float32') / 255.0
prediction = model.predict(np.expand_dims(test_img, axis=0))[0][0]
print("TNT Detected" if prediction > 0.5 else "No TNT Detected")



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 720ms/step
TNT Detected


![Screen Shot 2022-04-30 at 5.25.53 AM.png](attachment:5ffe6c33-f176-418f-b0c2-7de8fdb54ca2.png)

![image.png](attachment:3c0f5fe4-5dce-4309-9d13-6c0ca1456c23.png)

![image.png](attachment:4ed21fdd-484c-47ae-a521-408e48c1d204.png)

![Unknown.png](attachment:fe969990-2e4f-41ad-ba28-eddf83c2b9ed.png)


# classification model training



- Use cv2.threshold and cv2.dilate to create image masks

![Tiling 11slidew0slideh1024.png](attachment:47006a89-aecb-43af-b961-afd505921a20.png)

- Use transfer learning to create a classification model for images with and without TNTs
- You can start from and use VGG16
- Your goal is to run as many as models as the types of images you created in the preprocessing step, that is, if you created images of size 512 and 250 pixels, then you will have two models for each image size.